In [1]:
import pandas as pd
import glob
import difflib
import sys
sys.path.append('../../lib/')
import audlutils as au
import numpy as np

In [3]:
user_names = pd.read_csv('../../data/supplemental/username_playername_relation.csv',encoding = "ISO-8859-1")
all_team_names = user_names.Teamname.unique()
user_names['InRel']=True
user_names.head(1)

,Unnamed: 0,index,Teamname,Tournament,Username,PlayerName,Year,InRel
0,0,0,Atlanta Hustle,AUDL 2015,A Olsen,Anders Olsen,2015,True


In [5]:
raw_list=[]
for year in range(2014,2020):
    for raw_file in glob.glob(f'../../data/raw/{year}/*csv'):
        teamname = raw_file.split('_')[-1].split('.')[0]
        teamname = difflib.get_close_matches(teamname, all_team_names)[0]
        temp_df = au.csv2dataframe(raw_file)
        temp_df['Year']=year
        temp_df['Tournament']=f'AUDL {year}'
        temp_df['Teamname']=teamname
        raw_list.append(temp_df)

audl = pd.concat(raw_list)
audl.head(1)

,Date/Time,Tournamemnt,Opponent,Point Elapsed Seconds,Line,Our Score - End of Point,Their Score - End of Point,Event Type,Action,Passer,...,End Area,End X,End Y,Distance Unit of Measure,Absolute Distance,Lateral Distance,Toward Our Goal Distance,Year,Tournament,Teamname
0,2014-04-27 15:00,AUDL,Rochester Dragons,11,O,1,0,Offense,Catch,Sam T,...,,,,,,,,2014,AUDL 2014,DC Breeze


In [6]:
audl.columns

Index(['Date/Time', 'Tournamemnt', 'Opponent', 'Point Elapsed Seconds', 'Line',
       'Our Score - End of Point', 'Their Score - End of Point', 'Event Type',
       'Action', 'Passer', 'Receiver', 'Defender', 'Hang Time (secs)',
       'Player 0', 'Player 1', 'Player 2', 'Player 3', 'Player 4', 'Player 5',
       'Player 6', 'Player 7', 'Player 8', 'Player 9', 'Player 10',
       'Player 11', 'Player 12', 'Player 13', 'Player 14', 'Player 15',
       'Player 16', 'Player 17', 'Player 18', 'Player 19', 'Player 20',
       'Player 21', 'Player 22', 'Player 23', 'Player 24', 'Player 25',
       'Player 26', 'Player 27', 'Elapsed Time (secs)', 'Begin Area',
       'Begin X', 'Begin Y', 'End Area', 'End X', 'End Y',
       'Distance Unit of Measure', 'Absolute Distance', 'Lateral Distance',
       'Toward Our Goal Distance', 'Year', 'Tournament', 'Teamname'],
      dtype='object')

In [7]:
numbered_player_fields = [f'Player {i}' for i in range(0,28)]
player_fields = ['Passer', 'Receiver', 'Defender'] + numbered_player_fields

In [8]:
ugb = audl.groupby(['Teamname',
                  'Tournament',
                  'Year']).apply(lambda x : pd.DataFrame([{'Username':p} for p in np.unique([ plyr for field in player_fields 
                                                                                                   for plyr in x[field].unique() ])]) )

ugb = ugb.reset_index()
ugb['InRaw'] =True
merged = pd.merge(user_names,
                     ugb,
                     on = ['Teamname',
                              'Tournament',
                              'Year','Username'],
                     how='outer')

merged = merged[~merged.Username.isin(['','Anonymous'])]

In [10]:
actives = pd.read_csv('../../data/supplemental/weekly_active_rosters.csv')
actives.head(1)
all_actives = actives.groupby('Team').apply(lambda x : pd.Series({'AllActives':'; '.join(x.Actives).replace('**','')})).reset_index()
all_actives_dict = pd.Series(all_actives.AllActives.values,index=all_actives.Team.values).to_dict()
all_actives_dict['Atlanta Hustle']

'Sam Batson; Sun Choi; Karl Ekwurtzel; Brett Hulsmeyer; Elijah Jaime; Matthew Knowles; Paul Lally; Will Lindquist; Mac McClellan; Taylor Minch; Javier Ortiz; Player Pierce; Devon Rogers; Matt Smith; Trenton Spinks; Austin Taylor; Alex Trautman; Josh Turner; Kelvin Williams; Joel Wooten; Zach Avello; Josh Bush; Sun Choi; Jason Crowe; Karl Ekwurtzel; Brett Hulsmeyer; Elijah Jaime; Matthew Knowles; Paul Lally; Mac McClellan; Christian Olsen; Player Pierce; Carter Rae; Devon Rogers; Matt Smith; Trenton Spinks; Austin Taylor; Alex Trautman; Josh Turner; Kelvin Williams; Zack Avello; Josh Bush; Sun Choi; Jason Crowe; Phillip Dougherty; Karl Ekwurtzel; Matthew Knowles; Mac McClellan; Christian Olsen; Javier Ortiz; Jaime Perez; Player Pierce; Carter Rae; Matt Smith; Trenton Spinks; Austin Taylor; Alex Trautman; Josh Turner; Kelvin Williams; Joel Wooten; Inactive:\xa0Elijah Jaime, Brett Hulsmeyer; Zack Avello; Sam Batson; Sun Choi; Jason Crowe; Karl Ekwurtzel; Christian Gaeta; Matthew Knowles; 

In [11]:
difflib.get_close_matches('A Taylor',all_actives_dict.get('Atlanta Hustle','').split('; '))

['Austin Taylor', 'Austin Taylor', 'Austin Taylor']

In [13]:
import re
def ReverseIt(name):
    return ' '.join(re.findall('[A-Z][^A-Z]*',name)[::-1])

In [14]:
'FioDan',\
difflib.SequenceMatcher(None, 'FioDan', 'Dan Fiorino').ratio(), \
ReverseIt('FioDan'), \
difflib.SequenceMatcher(None, ReverseIt('FioDan'), 'Dan Fiorino').ratio(),

('FioDan', 0.47058823529411764, 'Dan Fio', 0.7777777777777778)

## Test for 2019 only

In [15]:
merged['InRel'] = merged['InRel'].fillna(False)
merged['InRaw'] = merged['InRaw'].fillna(False)


# unmatched = merged[(merged.InRaw)&~merged.InRel].copy() # in username but not raw
unmatched = merged[(merged.Year==2019)].copy() # in username but not raw

# Guess on username
unmatched['Guesses1'] = unmatched.apply(lambda x : difflib.get_close_matches(x.Username,
                                                                             all_actives_dict.get(x.Teamname,'').split('; '),cutoff=.001  ) ,
                                        axis=1)
unmatched['Guess1'] = unmatched['Guesses1'].apply(lambda x : x[0] if len(x)>0 else '')
unmatched['GuessScore1'] = unmatched.apply(lambda x : difflib.SequenceMatcher(None, x.Username, x.Guess1).ratio(),axis=1)

# Guess on Username with capitalized segments reversed

unmatched['Guesses2'] = unmatched.apply(lambda x : difflib.get_close_matches(ReverseIt(x.Username) ,
                                                                             all_actives_dict.get(x.Teamname,'').split('; '),cutoff=.001  ) ,
                                        axis=1)
unmatched['Guess2'] = unmatched['Guesses2'].apply(lambda x : x[0] if len(x)>0 else '')
unmatched['GuessScore2'] = unmatched.apply(lambda x : difflib.SequenceMatcher(None, ReverseIt(x.Username), x.Guess2).ratio(),
                                           axis=1)

# Guess on previous username matches
unmatched['Guesses3'] = unmatched.apply(lambda x : difflib.get_close_matches(x.Username ,
                                                                             user_names[(user_names.Teamname==x.Teamname)&(user_names.Username==x.Username)].PlayerName.dropna().values,cutoff=.001  ) ,
                                        axis=1)
unmatched['Guess3'] = unmatched['Guesses3'].apply(lambda x : x[0] if len(x)>0 else '')
unmatched['GuessScore3'] = unmatched.apply(lambda x : difflib.SequenceMatcher(None, x.Username, x.Guess3).ratio(),
                                           axis=1)
def GetBestGuess(row,thresh1=.51,thresh2=.51):
    """Get Best Guess"""
    if row['GuessScore1'] > thresh1:
        return row['Guess1']
    elif row['GuessScore2'] > thresh2:
        return row['Guess2']
    else:
        return row['Guess3']


unmatched['BestGuess'] = unmatched.apply(GetBestGuess,axis=1)
# unmatched.sort_values('GuessScore1')


## Full Guesses

In [16]:
merged['InRel'] = merged['InRel'].fillna(False)
merged['InRaw'] = merged['InRaw'].fillna(False)

# Guess on username
merged['Guesses1'] = merged.apply(lambda x : difflib.get_close_matches(x.Username,
                                                                             all_actives_dict.get(x.Teamname,'').split('; '),cutoff=.001  ) ,
                                        axis=1)
merged['Guess1'] = merged['Guesses1'].apply(lambda x : x[0] if len(x)>0 else '')
merged['GuessScore1'] = merged.apply(lambda x : difflib.SequenceMatcher(None, x.Username, x.Guess1).ratio(),axis=1)

# Guess on Username with capitalized segments reversed
merged['Guesses2'] = merged.apply(lambda x : difflib.get_close_matches(ReverseIt(x.Username) ,
                                                                             all_actives_dict.get(x.Teamname,'').split('; '),cutoff=.001  ) ,
                                        axis=1)
merged['Guess2'] = merged['Guesses2'].apply(lambda x : x[0] if len(x)>0 else '')
merged['GuessScore2'] = merged.apply(lambda x : difflib.SequenceMatcher(None, ReverseIt(x.Username), x.Guess2).ratio(),
                                           axis=1)

# Guess on previous username matches
merged['Guesses3'] = merged.apply(lambda x : difflib.get_close_matches(x.Username ,
                                                                         user_names[(user_names.Teamname==x.Teamname)
                                                                                    &(user_names.Username==x.Username)].PlayerName.dropna().values,cutoff=.001  ) ,
                                        axis=1)
merged['Guess3'] = merged['Guesses3'].apply(lambda x : x[0] if len(x)>0 else '')
merged['GuessScore3'] = merged.apply(lambda x : difflib.SequenceMatcher(None, x.Username, x.Guess3).ratio(),
                                           axis=1)




## Apply Best Guess and Review

In [17]:
def GetBestGuess(row,thresh1=.51,thresh2=.51):
    """Get Best Guess"""
    if row['GuessScore1'] > thresh1:
        return row['Guess1']
    elif row['GuessScore2'] > thresh2:
        return row['Guess2']
    else:
        return row['Guess3']


merged['BestGuess'] = merged.apply(GetBestGuess,axis=1)

merged[pd.isnull(merged.PlayerName)&(merged.BestGuess=='')]

cut = pd.isnull(merged.PlayerName)&(merged.Year==2019)
merged.loc[cut,'PlayerName']= merged[cut].BestGuess
merged[cut]


,Unnamed: 0,index,Teamname,Tournament,Username,PlayerName,Year,InRel,level_3,InRaw,Guesses1,Guess1,GuessScore1,Guesses2,Guess2,GuessScore2,Guesses3,Guess3,GuessScore3,BestGuess
3918,3918.0,3936.0,Austin Sol,AUDL 2019,Mika C,Corey Mika,2019,True,22.0,True,"[Michael Cramer, Michael Cramer, Michael Cramer]",Michael Cramer,0.500000,"[Corey Mika, Corey Mika, Corey Mika]",Corey Mika,0.705882,[],,0.000000,Corey Mika
3936,3936.0,3986.0,DC Breeze,AUDL 2019,Dennis M,Dennis Maclaine,2019,True,11.0,True,"[Dennis Maclaine, Dennis Maclaine, Dennis Macl...",Dennis Maclaine,0.695652,"[Dennis Maclaine, Dennis Maclaine, Dennis Macl...",Dennis Maclaine,0.583333,[],,0.000000,Dennis Maclaine
3940,3940.0,3990.0,DC Breeze,AUDL 2019,Kenta K,Kenta Kawaguchi,2019,True,21.0,True,"[Kenta Kawaguchi, Kenta Kawaguchi, Kevin Verst...",Kenta Kawaguchi,0.636364,"[Kenta Kawaguchi, Kenta Kawaguchi, Ray Mendoza]",Kenta Kawaguchi,0.521739,[],,0.000000,Kenta Kawaguchi
3941,3941.0,3991.0,DC Breeze,AUDL 2019,Kris H,,2019,True,23.0,True,"[Kristopher Harrison, Kristopher Harrison, Kri...",Kristopher Harrison,0.480000,"[Kristopher Harrison, Kristopher Harrison, Kri...",Kristopher Harrison,0.384615,[],,0.000000,
3957,3957.0,4015.0,Dallas Roughnecks,AUDL 2019,BRogers,Ben Rogers,2019,True,7.0,True,"[Ben Rogers, Ben Rogers, Alex Brouwer]",Ben Rogers,0.823529,"[Ben Rogers, Ben Rogers, Noah Chambers]",Ben Rogers,0.666667,[],,0.000000,Ben Rogers
3958,3958.0,4016.0,Dallas Roughnecks,AUDL 2019,BStreet,,2019,True,8.0,True,"[Stephen Poulos, Ben Lewis, Ben Lewis]",Stephen Poulos,0.380952,"[Stephen Poulos, Stephen Poulos, Wesley Miaw]",Stephen Poulos,0.454545,[],,0.000000,
3959,3959.0,4017.0,Dallas Roughnecks,AUDL 2019,CHogg,,2019,True,9.0,True,"[Connor Ughetta, Connor Ughetta, Connor Ugh...",Connor Ughetta,0.300000,"[Joel Clutton, Joel Clutton, Joel Clutton]",Joel Clutton,0.333333,[],,0.000000,
4106,4106.0,4334.0,Seattle Cascades,AUDL 2019,Edie,,2019,True,NaN,False,"[Eide Charlie, Zhi Chen, Zhi Chen]",Eide Charlie,0.500000,"[Eide Charlie, Zhi Chen, Zhi Chen]",Eide Charlie,0.500000,[],,0.000000,
4107,4107.0,4335.0,Seattle Cascades,AUDL 2019,Geertz,,2019,True,13.0,True,"[Peter Geertz-Larson, Peter Geertz-Larson, Pet...",Peter Geertz-Larson,0.480000,"[Peter Geertz-Larson, Peter Geertz-Larson, Pet...",Peter Geertz-Larson,0.480000,[],,0.000000,
4124,4124.0,4357.0,Tampa Bay Cannons,AUDL 2019,Allen,Brandon Allen,2019,True,2.0,True,"[Brandon Allen, Brandon Allen, Kellen Gray]",Brandon Allen,0.555556,"[Brandon Allen, Brandon Allen, Kellen Gray]",Brandon Allen,0.555556,[],,0.000000,Brandon Allen


## Save as Updated Version

In [18]:
merged[['Teamname', 'Tournament', 
        'Username', 'PlayerName', 
        'Year']].reset_index().to_csv('../../data/supplemental/username_playername_relation_updated.csv')

In [21]:
len(merged),len(user_names),merged.columns,len(merged[[ 'Teamname', 'Tournament', 'Username', 'PlayerName', 'Year']].drop_duplicates())

(4578,
 4148,
 Index(['Unnamed: 0', 'index', 'Teamname', 'Tournament', 'Username',
        'PlayerName', 'Year', 'InRel', 'level_3', 'InRaw', 'Guesses1', 'Guess1',
        'GuessScore1', 'Guesses2', 'Guess2', 'GuessScore2', 'Guesses3',
        'Guess3', 'GuessScore3', 'BestGuess'],
       dtype='object'),
 4578)

## Review Unmatched

In [27]:
merged[merged.PlayerName==''][['Teamname', 'Tournament', 'Username', 'PlayerName', 'Year','Guess1','Guess2','Guess3']]

,Teamname,Tournament,Username,PlayerName,Year,Guess1,Guess2,Guess3
3941,DC Breeze,AUDL 2019,Kris H,,2019,Kristopher Harrison,Kristopher Harrison,
3958,Dallas Roughnecks,AUDL 2019,BStreet,,2019,Stephen Poulos,Stephen Poulos,
3959,Dallas Roughnecks,AUDL 2019,CHogg,,2019,Connor Ughetta,Joel Clutton,
4106,Seattle Cascades,AUDL 2019,Edie,,2019,Eide Charlie,Eide Charlie,
4107,Seattle Cascades,AUDL 2019,Geertz,,2019,Peter Geertz-Larson,Peter Geertz-Larson,
4126,Tampa Bay Cannons,AUDL 2019,Barrett,,2019,Jack Barrow,Jack Barrow,
4160,Atlanta Hustle,AUDL 2019,Frolich,,2019,Taylor Minch,Taylor Minch,
4161,Atlanta Hustle,AUDL 2019,Gaeta,,2019,Christian Gaeta,Christian Gaeta,
4162,Atlanta Hustle,AUDL 2019,Hayden,,2019,Hayden Austin-Knabb,Hayden Austin-Knabb,
4182,Austin Sol,AUDL 2019,Darroh,,2019,Carter Hollo,Carter Hollo,


In [23]:
merged[pd.isnull(merged.PlayerName)][['Teamname', 'Tournament', 'Username', 'PlayerName', 'Year']]

,Teamname,Tournament,Username,PlayerName,Year
126,Atlanta Hustle,AUDL 2016,The Law,NaN,2016
136,Atlanta Hustle,AUDL 2016,Walsh,NaN,2016
243,Austin Sol,AUDL 2018,Unknown,NaN,2018
324,Chicago Wildfire,AUDL 2018,Bruno,NaN,2018
329,Chicago Wildfire,AUDL 2018,Chowder,NaN,2018
333,Chicago Wildfire,AUDL 2015,DW,NaN,2015
377,Chicago Wildfire,AUDL 2015,Kennedy,NaN,2015
395,Chicago Wildfire,AUDL 2018,Nick,NaN,2018
500,Cincinnati Revolution,AUDL 2015,Greg,NaN,2015
727,Dallas Roughnecks,AUDL 2018,Blake,NaN,2018


In [24]:
user_names[pd.isnull(user_names.PlayerName)][['Teamname', 'Tournament', 'Username', 'PlayerName', 'Year']]

,Teamname,Tournament,Username,PlayerName,Year
126,Atlanta Hustle,AUDL 2016,The Law,NaN,2016
136,Atlanta Hustle,AUDL 2016,Walsh,NaN,2016
243,Austin Sol,AUDL 2018,Unknown,NaN,2018
324,Chicago Wildfire,AUDL 2018,Bruno,NaN,2018
329,Chicago Wildfire,AUDL 2018,Chowder,NaN,2018
333,Chicago Wildfire,AUDL 2015,DW,NaN,2015
377,Chicago Wildfire,AUDL 2015,Kennedy,NaN,2015
395,Chicago Wildfire,AUDL 2018,Nick,NaN,2018
500,Cincinnati Revolution,AUDL 2015,Greg,NaN,2015
727,Dallas Roughnecks,AUDL 2018,Blake,NaN,2018


## Test Travis Dunn 2019

In [29]:
up = pd.read_csv('../../data/supplemental/username_playername_relation.csv',encoding = "ISO-8859-1")

up[up.PlayerName=='Travis Dunn']

,Unnamed: 0,Teamname,Tournament,Username,PlayerName,Year
2930,2930,San Diego Growlers,AUDL 2016,Dunn,Travis Dunn,2016
2931,2931,San Diego Growlers,AUDL 2017,Dunn,Travis Dunn,2017
2932,2932,San Diego Growlers,AUDL 2018,Dunn,Travis Dunn,2018
2933,2933,San Diego Growlers,AUDL 2015,Dunn T,Travis Dunn,2015
4054,4054,San Diego Growlers,AUDL 2019,Dunn,Travis Dunn,2019
4075,4075,San Diego Growlers,AUDL 2019,Tran,Travis Dunn,2019
